## Data preparation from websites

In [1]:
import re
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


import pandas as pd
from bs4 import BeautifulSoup

import string
import time

import warnings
warnings.filterwarnings('ignore')


#print multiple statements in same line
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ="all"

In [2]:
import re
def extractURLs(fileContent):
    urls = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', fileContent.lower())
    cleanUrls = []
    for url in urls:
        lastChar = url[-1] # get the last character
        # if the last character is not (^ - not) an alphabet, or a number,
        # or a '/' (some websites may have that. you can add your own ones), then enter IF condition
        if (bool(re.match(r'[^a-zA-Z0-9/]', lastChar))): 
            cleanUrls.append(url[:-1]) # stripping last character, no matter what
        else:
            cleanUrls.append(url) # else, simply append to new list
    #print(cleanUrls)
    return cleanUrls

In [3]:
pContentList = []
import re
linkList = [] 
with open("datasets\Weblink.txt") as file:
        for line in file:
            urls = extractURLs(line)
            for eachlink in urls:
              session = requests.Session()
              retry = Retry(connect=5, backoff_factor=0.5)
              time.sleep(0.2)
              adapter = HTTPAdapter(max_retries=retry)
              session.mount('http://', adapter)
              session.mount('https://', adapter)
              pageObject  = session.get(eachlink)
              soupObject  = BeautifulSoup(pageObject.content, 'html.parser')
              divPtags = soupObject.findAll('div', href = False)
              allPtags = soupObject.findAll('p', href = False)
            for eachPtag in allPtags:
                actual_content  = str(eachPtag.getText())      
                pContentList.append(actual_content)
            for eachPtag in divPtags:
                actual_content1  = str(eachPtag.getText())      
                pContentList.append(actual_content1)

In [4]:
tileDataFrame = pd.DataFrame(pContentList, columns=['Content'])
tileDataFrame.shape

(1514, 1)

In [5]:
tileDataFrame.drop_duplicates(subset ="Content",keep = False, inplace = True)
tileDataFrame.shape

(289, 1)

In [6]:
tileDataFrame

,Content
164,In this section: \n
165,Diabetes is a disease that occurs when your bl...
166,"Over time, having too much glucose in your blo..."
167,Sometimes people call diabetes “a touch of sug...
168,"The most common types of diabetes are type 1, ..."
...,...
1330,\nIn this section:\n\nWhat foods can I eat if ...
1333,\nAdditional Links\n\nYoung at Heart: Tips for...
1334,"\n\n\nPrevious:\r\n Insulin, Me..."
1335,"\nPrevious:\r\n Insulin, Medici..."


In [7]:
tileDataFrame.to_csv('datasets\dataFinal1.csv')